In [1]:
import numpy as np 
from sklearn.model_selection import RepeatedKFold
import nibabel as nib
from scipy.io import loadmat

In [2]:
from sklearn.model_selection import train_test_split
import os, sys
import numpy as np
import pdb
import argparse
import json 
import subprocess
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

## Saliency

In [3]:
import utils as ut 
from models import AN3Ddr_lowresMax
import torch

In [ ]:
mdpath = '../out/results/latest/mt_AN3DdrlrMx_fkey_lT1_scorename_labels_3way_iter_10_nc_2_rep_0_bs_32_lr_0.0001_espat_20/model_state_dict.pt'

model = AN3Ddr_lowresMax(num_classes=2, num_channels=1, num_groups=1)
model.load_state_dict(torch.load(mdpath))


In [ ]:
d = '/data/users2/ibatta/data/features/fmrimeasures/ADNI/nii/002_S_6007/Axial_rsfMRI__Eyes_Open_/2017-03-31_10_40_49.0/S551365/rest/'

f1 = nib.load(d+'swarest1_tsavg.nii').get_fdata()
f2 = nib.load(d+'swarest1_ALFF.nii').get_fdata()


In [ ]:
np.prod(f1.shape)

In [ ]:
(f1-f2 > 10).sum()

In [ ]:
f1 = (f1 - f1.min()) / (f1.max()-f1.min())
f2 = (f2 - f2.min()) / (f2.max()-f2.min())


In [ ]:
(f1-f2 > 0.01).sum()

In [ ]:
fl = '/data/qneuromark/Data/ADNI/Updated/fMRI/ADNI/177_S_6335/Axial_MB_rsfMRI__Eyes_Open_/2020-07-22_12_56_01.0/S951207/rest/swarest1.nii'
dl = nib.load(fl).get_fdata()


In [ ]:
basedir='/data/qneuromark/Data/ADNI/Updated/fMRI/Results/GIGICA/041_S_6401/Axial_rsfMRI__Eyes_Open_/2018-06-12_13_51_03.0/S694594/rest/swarest1/'
f1 = 'adni_aa__sub01_component_ica_s1_.nii'
f2 = 'adni_aa__ica_c1-1.mat'



In [ ]:
d1 = nib.load(basedir + f1).get_fdata()

In [ ]:
d2 = loadmat(basedir + f2)

In [ ]:
d2.keys()

In [ ]:
d1.shape

In [ ]:
d2['ic'].shape, d2['tc'].shape

In [ ]:
53*63*52, 66529*194

##  QC 

### Constant Valued Scans

In [ ]:
dataset = 'ADNI'
filemapper = json.load(open('../in/filemapper.json','r'))

measures = list(filemapper['filename']['ADNI'].keys())

nanfiles = []
numnans = []
badfiles = []

for m in measures: 
    basepath_mapper = filemapper['basepathmapper'][dataset][m]
    basedir = filemapper['basedir'][dataset][basepath_mapper]
    # basedir = '/data/qneuromark/Data/ADNI/Updated/fMRI/ADNI/'
    filename = filemapper['filename']['ADNI'][m]
    # filename = 'swarest1.nii'
    for dir in open('../in/%s_MMR180.csv'%basepath_mapper,'r+').read().split('\n')[:-1]:
        fullpath = basedir + dir + filename
        scandata = nib.load(fullpath).get_fdata()
        if np.isnan(scandata).sum() > 0:
            nn = np.isnan(scandata).sum()
            print(nn)
            print(fullpath)
            nanfiles.append(fullpath)
            numnans.append(nn)
        if np.abs(np.std(scandata)) < 0.001:
            print(np.abs(np.std(scandata)))
            print(fullpath)
            badfiles.append(fullpath)




In [ ]:
with open('../in/badfiles_measures.txt','w+') as f:
    f.write('\n'.join(badfiles))

with open('../in/numnans_measures.txt','w+') as f:
    f.write('\n'.join(['%s,%d'%(nanfiles[i],numnans[i]) for i in range(len(nanfiles))]))
    

In [ ]:
len(['ALFF', 'DCw', 'DCb', 'fALFF', 'KccReHo', 'VMHC', 'lT1', 'ALFF,DCw,DCb,fALFF,KccReHo,VMHC,lT1', 'ALFF,lT1', 'DCw,lT1', 'DCb,lT1', 'fALFF,lT1', 'KccReHo,lT1', 'VMHC,lT1'])

### Imputations

In [ ]:
with open('../in/vars.txt', 'r+') as f:
    scores = f.read().split('\n')[:-1]
scores_file = '../in/analysis_SCORE_MMR180d.csv'
df = pd.read_csv(scores_file)


In [ ]:
print(df['age'].dtype)

In [ ]:

for score in scores:
    print('%s, %s'%(score,df[score].dtype))
    dt = df[score].dtype
    if not 'float' in str(dt):
        df[score] = df[score].fillna(df[score].mode())
    else:
        df[score] = df[score].fillna(df[score].median())

df.to_csv('../in/analysis_SCORE_MMR180d_imputed.csv',  index=False)

In [ ]:
output_file = '../out/performances/baseline/allcombos_baseline_reg.png '

In [ ]:
pval_file = '.'.join(output_file.split('.')[:-1]) + '_pvals.' + output_file.split('.')[-1]

In [ ]:
fns = ['tsavg','tsmedian','tsmax','tsmin']
    # fns.remove('hT1')
    # fns.remove('PerAF')
    # fns.append(','.join(fns)) #multimodal with all features into DL model
    fns += [fi+',lT1' for fi in fns if 'lT1' not in fi] # multimodal with 2 modalities: with one fmri measure with low-res smri 
    # fns.append(','.join([','.join(fns) for i in range(8)] )) # Testing 56 groups for future  
    print(fns)

    # # Map iter value (slurm taskID) to training sample size (tss) and crossvalidation repetition (rep)
    fv, rv = np.meshgrid(np.arange(len(fns)), np.arange(cfg.nReps))
    fv = fv.reshape((1, np.prod(fv.shape)))
    rv = rv.reshape((1, np.prod(rv.shape)))
    fkey = fns[fv[0][cfg.iter]]
    rep = rv[0][cfg.iter]
    print(fkey, rep)
    cfg.fkey = fkey
    cfg.nch = len(fkey.split(','))
    cfg.rep = rep
    print(cfg.iter, cfg.tss, cfg.rep)


In [ ]:
fm = json.load(open('../in/filemapper.json','r'))

In [ ]:
fns = list(fm['filename']['ADNI'].keys())

In [ ]:
len(fns)

In [ ]:
fns = ['tsavg','tsmedian','tsmax','tsmin']
# fns.remove('hT1')
# fns.remove('PerAF')
# fns.append(','.join(fns)) #multimodal with all features into DL model
fns += [fi+',lT1' for fi in fns if 'lT1' not in fi] # multimodal with 2 modalities: with one fmri measure with low-res smri 

In [ ]:
i = 8
fv, rv, sv = np.meshgrid(np.arange(len(fns)), np.arange(10), np.arange(8))
fv = fv.reshape((1, np.prod(fv.shape)))
rv = rv.reshape((1, np.prod(rv.shape)))
sv = sv.reshape((1, np.prod(sv.shape)))

for i in range(80):
    fkey = fns[fv[0][i]]
    rep = rv[0][i]
    scr = sv[0][i]
    print(fkey, rep, scr)

In [ ]:
fv.shape, rv.shape, sv.shape

In [ ]:
i = 8
fns.remove('hT1')
fns.remove('PerAF')

fv, rv = np.meshgrid(np.arange(len(fns)), np.arange(10))
fv = fv.reshape((1, np.prod(fv.shape)))
rv = rv.reshape((1, np.prod(rv.shape)))

for i in range(80):
    fkey = fns[fv[0][i]]
    rep = rv[0][i]
    
    print(i,fkey, rep)

In [ ]:
def summarize_config(config_string):
    """
    Computes the set of test accuracy for the string corresponding to the config_string. 
    The config strings should represent one of the directory. 
    """

    basedir = '../out/results/latest/'
    cmd = 'cat %s%s/test.csv | grep -v acc_te | cut -f2 -d\",\"'%(basedir,config_string)
    output_stream = os.popen(cmd)
    test_accs = np.array(output_stream.read().split('\n')[:-1], dtype=float)
    return test_accs

In [ ]:
config_string = 'AN3Ddr_lowresMax_fkey_ALFF_scorename_labels_3way_iter_*_nc_2_rep_*_bs_32_lr_1e-05_espat_20'

In [ ]:
t = summarize_config(config_string)

In [ ]:
len(t)

In [ ]:
t = summarize_config('AN3Ddr_lowresMax_fkey_ALFF_scorename_labels_3way_iter_*_nc_2_rep_*_bs_32_lr_1e-05_espat_20')

In [ ]:

n = 30
k_te = 4
k_va = 5
nreps = 3

all_labels = np.array(list(range(0,30)), dtype=int) // 10

# selelct specific labels if needed for binary classification
select_labels = [0,2]
labels = np.array([li for li in all_labels if li in select_labels], dtype=int)
sli = np.array([i for i in range(len(all_labels)) if all_labels[i] in select_labels], dtype=int)


for ri in np.arange(nreps):
	tri, test_indices = train_test_split(np.arange(len(labels)), test_size=1.0/k_te, shuffle=True, stratify=labels, random_state=108+ri)
	tr_idx, val_idx = train_test_split(np.arange(len(tri)), test_size=1/k_va, shuffle=True, stratify=labels[tri], random_state=108+ri)
	train_indices, val_indices = tri[tr_idx], tri[val_idx]

	print(all_labels[sli[train_indices]], all_labels[sli[val_indices]], all_labels[sli[test_indices]])
    


In [ ]:
np.loadtxt('../out/temp1.txt').mean()

In [ ]:
np.loadtxt('../out/temp2.txt').mean()

In [ ]:


	np.savetxt(outdir+'/tr_r'+str(ri)+'.csv', train_indices, fmt='%d')
	np.savetxt(outdir+'/va_r'+str(ri)+'.csv', val_indices, fmt='%d')
	np.savetxt(outdir+'/te_r'+str(ri)+'.csv', test_indices, fmt='%d')



In [ ]:
from sklearn.model_selection import train_test_split

tr, va = train_test_split(np.arange(n), test_size=0.2)

In [ ]:
tr, va

In [ ]:
import pandas as pd 
# Try Merge
fl = '../in/lowresfilelist_ADNI_SMRI.txt'
sm = '../in/analysis_SCORE_SMRI_fpL.csv'

df_fl = pd.read_csv(fl)
df_sm = pd.read_csv(sm).iloc[idx]


In [ ]:
for iter in range(12):
    tv, rv = np.meshgrid([], np.arange(3))
    tv = tv.reshape((1, np.prod(tv.shape)))
    rv = rv.reshape((1, np.prod(tv.shape)))
    tss = tv[0][iter]
    rep = rv[0][iter]
    print(tss, rep)

In [ ]:
import nibabel as nib 

In [ ]:
df_sm

In [ ]:
df_sm

In [ ]:
idx = np.loadtxt('../in/SampleSplits/ADNI/3way/te_r0.csv', dtype=int)

In [ ]:
idx

In [ ]:
df_sm = df_sm.iloc[idx]

In [ ]:
df_sm